In [2]:
#clear all
%reset -f

#import packages
import numpy as np
import sys
import os
import pandas as pd
import mne
import matplotlib
from sklearn.utils import resample
from mne_icalabel import label_components

root = 'F:/Desktop/MirRevAdapt_EEG/'
participants = list(range(0,32))

#pop up plots as separate window & interactive
%matplotlib qt
matplotlib.pyplot.close('all')

In [3]:
#setting up path/ directory

#access specific file for specific participant
def load_epochs(pp_num, root_dir):

    root_directory = root_dir
    data_directory = os.path.join(root_directory, 'data/mra/eeg/')
    pp_directory = os.path.join(data_directory, 'p%03d/' % pp_num)
    filename = os.path.join(pp_directory, 'mra_p%03d_run0-epo.fif' % pp_num)

    epochs = mne.read_epochs(filename)
    #baseline for each trial
    epochs = epochs.apply_baseline(baseline = (-1.5, -1.2))

    return epochs, pp_directory

In [4]:
def rotate_trials(task, pp, epochs):

    if pp % 2 == 0:
        if task == 'rot':
            #get trial after first two tasks (trial 96 to 185)
            epochs = epochs[96:186]
        elif task == 'rdm':
            #get trial after first task (trial 48 to 95)
            epochs = epochs[48:96]
    elif pp % 2 == 1:
        if task == 'rot':
            #get trial after first five tasks (trial 282 to 371)
            epochs = epochs[282:372]
        elif task == 'rdm':
            #get trial after first five tasks (trial 234 to 281)
            epochs = epochs[234:282]
    
    return(epochs)

In [5]:
def epoch_index(task, listval, pp, epochs):
    if task == 'rot':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ROT_learningcurve_degrees.csv')
    elif task == 'rdm':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\RDMROT_learningcurve_degrees.csv')
    elif task == 'aln':
        df = pd.read_csv(r'F:\Documents\Science\MirRevAdaptEEG\data\ALIGNED_learningcurve_degrees.csv')
        
    ppdata = list(df.iloc[:,pp+1])
    
    index = [i for i,v in enumerate(ppdata) if listval[0] < v < listval[1] or listval[2] < v < listval[3]]
    
    epochs_out = epochs[index]
    numtrials = len(index)
    
    return epochs_out, numtrials

In [6]:
def output_condition_epochs(pp_num, data, ppdir, condname, task):
    # Save the cleaned data :)
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s-epo.fif' % (pp_num, condname, task))
    data.save(out_fname, overwrite = True)

In [7]:
def output_condition_evoked(pp_num, data, ppdir, condname, task):
    evoked = data.average()
    out_fname = os.path.join(ppdir, 'p%03d_%s_%s-ave.fif' % (pp_num, condname, task))
    evoked.save(out_fname, overwrite = True)

In [8]:
tasknames = ['rot', 'rdm']
# percentage to split the data into different error size (change if using degrees)
conditions = [[15, 25, 35, 45], [-15, 15, 45, 75]]
# conditions = [[70, 100, 100, 130], [40, 70, 130, 160], [10, 40, 160, 190]]
# conditions = [[50, 100, 100, 150], [0, 50, 150, 200]]
conditionnames = ['small', 'lrg']
# conditionnames = ['smallPercent', 'lrgPercent']
#participants.remove(18)
l_cutoff = 30
#within 5 degrees is considered a "hit"
err_cutoff = [-5, 0, 0, 5]



#aligned
for pp in participants:
    data, ppdir = load_epochs(pp, root_dir = root)
    aligned_epochs = data[0:48]
    aldata, numtrials = epoch_index('aln', err_cutoff, pp, aligned_epochs)
    #print(pp, numtrials)
    output_condition_epochs(pp, aldata, ppdir, 'aligned', 'hits')
    #filter at 30 Hz
    aldata = aldata.filter(l_freq= None, h_freq=l_cutoff)
    #then average across trials
    output_condition_evoked(pp, aldata, ppdir, 'aligned', 'hits')

# df = pd.DataFrame()
# counter = 0

#rot and rdm
for task in range(0, len(tasknames)):
    for pp in participants:
        data, ppdir = load_epochs(pp, root_dir = root)
        epochs = rotate_trials(tasknames[task], pp, data)
        for cond in range(0, len(conditions)):
            data, numtrials = epoch_index(tasknames[task], conditions[cond], pp, epochs)
            output_condition_epochs(pp, data, ppdir, conditionnames[cond], tasknames[task])
            #filter at 30 Hz
            data = data.filter(l_freq= None, h_freq=l_cutoff)
            #then average across trials
            output_condition_evoked(pp, data, ppdir, conditionnames[cond], tasknames[task])
#             print('Participant:', pp, conditionnames[cond], ':', numtrials)
#             df.loc[counter, "participant"] = pp
#             df.loc[counter, "task"] = tasknames[task]
#             df.loc[counter, "condition"] = conditionnames[cond]
#             df.loc[counter, "ntrials"] = numtrials
            
#             counter += 1
            
# df_filename = os.path.join(os.getcwd(), 'alldat.csv')
# df.to_csv(df_filename)

Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p000\mra_p000_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p001\mra_p001_run0-epo.fif ...


[Parallel(n_jobs=1)]: Done 2560 out of 2560 | elapsed:    0.4s finished


    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



...

Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.0s remaining:    0.0s


Overwriting existing file.
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p014\mra_p014_run0-epo.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms
        0 CTF compensation matrices available
Not setting metadata
420 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Overwriting existing file.
Setting up low-pass filter at 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal lowpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 45 samples (0.450 sec)



[Parallel(n_jobs=1)]: Done 2880 out of 2880 | elapsed:    0.4s finished
C:\Users\Raphael\AppData\Local\Temp\ipykernel_2416\104609171.py:4: RuntimeWarning: Saving epochs with no data
  data.save(out_fname, overwrite = True)


ValueError: No appropriate channels found for the given picks (array([], dtype=int32))

In [9]:
root_directory = root
data_directory = os.path.join(root_directory, 'data/mra/eeg/')

#read in evoked object
tasknames = ['rot', 'rdm']
conditionnames = ['small', 'lrg']

grand_aligned = []
grand_small = []
grand_lrg = []

#aligned
flist = []
evoked_list = []

for pp in participants:
    pp_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
    fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], 'aligned', 'hits'))
    evoked = mne.read_evokeds(fname)
    flist.append(evoked)
    evoked = flist[pp][0]
    evoked_list.append(evoked)
        
grand_aligned = mne.grand_average(evoked_list) 
aligned_flist = evoked_list
    
#rot and rdm
for task in range(0, len(tasknames)):
    for condition in range(0, len(conditionnames)):
        flist = []
        evoked_list = []
        for pp in participants:
            pp_directory = os.path.join(data_directory, 'p%03d/' % participants[pp])
            fname = os.path.join(pp_directory, 'p%03d_%s_%s-ave.fif' % (participants[pp], conditionnames[condition], tasknames[task]))
            evoked = mne.read_evokeds(fname)
            flist.append(evoked)
            evoked = flist[pp][0]
            evoked_list.append(evoked)
            if task == 0 and condition == 0:
                grand_rot_small = mne.grand_average(evoked_list)
                small_rot_flist = evoked_list
            elif task == 0 and condition == 1:
                grand_rot_lrg = mne.grand_average(evoked_list)
                lrg_rot_flist = evoked_list
            elif task == 1 and condition == 0:
                grand_rdm_small = mne.grand_average(evoked_list)
                small_rdm_flist = evoked_list
            elif task == 1 and condition == 1:
                grand_rdm_lrg = mne.grand_average(evoked_list)
                lrg_rdm_flist = evoked_list
            

Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p000\p000_aligned_hits-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 40 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p001\p001_aligned_hits-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 28 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p002\p002_aligned_hits-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrice

        nave = 35 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p021\p021_aligned_hits-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 33 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p022\p022_aligned_hits-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 35 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Reading

C:\Users\Raphael\AppData\Local\Temp\ipykernel_2416\3825349473.py:40: RuntimeWarning: Only a single dataset was passed to mne.grand_average().
  grand_rot_small = mne.grand_average(evoked_list)


    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 43 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p009\p009_small_rot-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 41 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p010\p010_small_rot-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave 

        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 32 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p028\p028_small_rot-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 58 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p029\p029_small_rot-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 40 - aspect type = 100
No proj

...

        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 4 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p009\p009_small_rdm-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 3 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p010\p010_small_rdm-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 2 - aspect type = 100
No project

        0 CTF compensation matrices available
        nave = 11 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p028\p028_small_rdm-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 10 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p029\p029_small_rdm-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 3 - aspect type = 100
No projector specified for this dataset. Please consider 

C:\Users\Raphael\AppData\Local\Temp\ipykernel_2416\3825349473.py:49: RuntimeWarning: Only a single dataset was passed to mne.grand_average().
  grand_rdm_lrg = mne.grand_average(evoked_list)


        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 34 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p009\p009_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 38 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p010\p010_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 41 - aspect type = 100
No projecto

        nave = 27 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p028\p028_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 30 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is baseline-corrected (baseline: [-1.5, -1.2] sec)
Identifying common channels ...
Reading F:\Desktop\MirRevAdapt_EEG\data\mra\eeg\p029\p029_lrg_rdm-ave.fif ...
    Found the data of interest:
        t =   -1500.00 ...    1490.00 ms (16142)
        0 CTF compensation matrices available
        nave = 40 - aspect type = 100
No projector specified for this dataset. Please consider the method self.add_proj.
Loaded Evoked data is b

In [ ]:
#convert evokeds to data frame, so we can plot the ERPs better

flists = ['aln', 'smlrot', 'lrgrot', 'smlrdm', 'lrgrdm']

for f in flists:
    newdf = pd.DataFrame()
    for pp in participants:
        if f == 'aln':
            df = aligned_flist[pp].to_data_frame()
        elif f == 'smlrot':
            df = small_rot_flist[pp].to_data_frame()
        elif f == 'lrgrot':
            df = lrg_rot_flist[pp].to_data_frame()
        elif f == 'smlrdm':
            df = small_rdm_flist[pp].to_data_frame()
        elif f == 'lrgrdm':
            df = lrg_rdm_flist[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        ppdat = []
        for timept in range(0,300):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Evoked_DF_%s.csv' % f)
    newdf.to_csv(newdf_filename)
        

In [36]:
#plot ERP
#channels = ['FCz', 'Fz', 'FC1', 'FC2', 'Cz']
channels = ['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']
# channels = ['Oz','O1','O2']
evokeds = dict(aligned = aligned_flist, 
               small_rot = small_rot_flist,
               lrg_rot = lrg_rot_flist, 
               #small_rdm = small_rdm_flist, 
               lrg_rdm = lrg_rdm_flist)
mne.viz.plot_compare_evokeds(evokeds, picks=channels, combine='mean', legend = 'lower right', ylim=dict(eeg=[-15, 5]), ci=True)

More than 6 channels, truncating title ...
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


[<Figure size 800x600 with 1 Axes>]

In [23]:
#calculate difference waves
diffconds = ['smlrot', 'lrgrot', 'lrgrdm']
smrot_diff = []
lgrot_diff = []
lgrdm_diff = []

for cond in range(0, len(diffconds)):
    for pp in participants:
        if cond == 0:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['small_rot'][pp]], 
                                                            weights=[-1, 1])
            smrot_diff.append(diffevks)
        elif cond == 1:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['lrg_rot'][pp]], 
                                                         weights=[-1, 1])
            lgrot_diff.append(diffevks)
        elif cond == 2:
            diffevks = mne.combine_evoked([evokeds['aligned'][pp], evokeds['lrg_rdm'][pp]], 
                                                         weights=[-1, 1])
            lgrdm_diff.append(diffevks)
    


In [50]:
#saved difference waves as data frame

difflists = ['smlrot', 'lrgrot', 'lrgrdm']

for diff in difflists:
    newdf = pd.DataFrame()
    for pp in participants:
        if diff == 'smlrot':
            df = smrot_diff[pp].to_data_frame()
        elif diff == 'lrgrot':
            df = lgrot_diff[pp].to_data_frame()
        elif diff == 'lrgrdm':
            df = lgrdm_diff[pp].to_data_frame()
             
        time = df['time'].tolist()
        df = df[['FCz', 'F3', 'Fz', 'F4', 'C3', 'Cz', 'C4', 'P3', 'Pz', 'P4']]
        ppdat = []
        for timept in range(0,300):
            subdf = df.loc[timept,:].mean()
            ppdat.append(subdf)
    
        newdf['pp%03d'% pp] = ppdat
    
    newdf['time'] = time
    newdf_filename = os.path.join('F:/Documents/Science/MirRevAdaptEEG/data/', 'Diffwaves_DF_%s.csv' % diff)
    newdf.to_csv(newdf_filename)
        


In [51]:
#plot difference waves
mne.viz.plot_compare_evokeds({'Smlrot - Aligned':smrot_diff, 'Lrgrot - Aligned':lgrot_diff, 'Lrgrdm - Aligned':lgrdm_diff}, 
                             picks=channels, show_sensors='upper right',
                             combine='mean',
                             title='Difference Wave');

combining channels using "mean"
combining channels using "mean"
combining channels using "mean"


In [45]:
# #plot topography?
# mne.viz.plot_evoked_topomap(mne.grand_average(smrot_diff), 
#                             times=.500, average=0.400, 
#                             title='SmallRot - Aligned',
#                             size=3
#                            )
# matplotlib.pyplot.show()

Identifying common channels ...


C:\Users\Raphael\AppData\Local\Temp\ipykernel_2416\1014179742.py:2: FutureWarning: The "title" parameter is deprecated and will be removed in version 1.3. Use "fig.suptitle()" instead.
  mne.viz.plot_evoked_topomap(mne.grand_average(smrot_diff),


In [ ]:
# list1 = [1, 2, 3, 4, 5, 6] 
# times = range(0,5)
# dat = []
# for time in times:
#     d = resample(list1, replace = True, n_samples = 10)
#     dat.append(d)
    
# print(dat)